### Create CSV from midi sources

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import json
import music21
from fastai.text import *
import glob
import os
from tqdm import tqdm
from IPython.display import Image, Audio
import traceback

In [3]:
from midi_data import *

In [4]:
path = Path('../data/midi')

In [5]:
directories = [x for x in path.iterdir() if x.is_dir()]; directories

[PosixPath('../data/midi/cprato_midi'),
 PosixPath('../data/midi/Wikifonia'),
 PosixPath('../data/midi/transposed'),
 PosixPath('../data/midi/hymn'),
 PosixPath('../data/midi/classic_piano_scrape'),
 PosixPath('../data/midi/touhou_scrape'),
 PosixPath('../data/midi/hooktheory'),
 PosixPath('../data/midi/ecomp_midi'),
 PosixPath('../data/midi/freemidi_scrape'),
 PosixPath('../data/midi/piano_e_competition'),
 PosixPath('../data/midi/midiworld_scrape')]

## General stream parsing

In [6]:
def get_music21_attr(fp, transpose=True, offset=None):
    try:
        stream = music21.converter.parse(fp)
    except Exception as e:
        print('Could not parse stream', fp, e)
#         traceback.print_exc()
        return {}
    stream_attr = get_stream_attr(stream)
    if transpose:
        transposed_file = transpose_midi2c(fp, stream, halfsteps=offset)
        transposed_stream = music21.converter.parse(fp)
        t_key = transposed_stream.analyze('key')
        transposed_attr = {
            'inferred_keyc': f'{t_key.tonic.name} {t_key.mode}',
            'midi_keyc': str(transposed_file),
        }
        stream_attr = {**stream_attr, **transposed_attr}
    return stream_attr 

In [7]:
def get_stream_attr(s):
    instruments = [i.instrumentName for i in list(s.getInstruments(recurse=True)) if i.instrumentName]
    metronome = list(filter(lambda x: isinstance(x, music21.tempo.MetronomeMark), s.flat))[0]
    bpm = metronome.getQuarterBPM()
    key = s.analyze('key')
    s_flat = s.flat
    time_sig = s_flat.timeSignature.ratioString if hasattr(s_flat.timeSignature, 'ratioString') else None
    return {
        'instruments': instruments,
        'bpm': bpm,
        'inferred_key': f'{key.tonic.name} {key.mode}',
        'seconds': s_flat.seconds,
        'time_signature': time_sig,
    }

### Gather Hooktheory Data

In [8]:
hook_path = path/'hooktheory'
hook_info_path = hook_path/'key2info.json'
hook_metapath = hook_path/'song_metadata.json'

def song_key(s): return '_'.join(s.parts[-3:-1])

In [9]:
key2info = load_json(hook_info_path)

if key2info is None:
    song_info = list((hook_path/'xml').glob('*/*/*/*.json'))
    key2info = {song_key(s):json.load(open(s, 'r')) for s in song_info}
    save_json(key2info, hook_info_path)
    len(song_info)

In [10]:
# song_xml = list((hook_path/'xml').glob('*/*/*/*.xml')); len(song_xml)
song_json = list((hook_path/'event').glob('*/*/*/*_key.json')); len(song_json)
# song_mid = list((hook_path/'pianoroll').glob('*/*/*/*_nokey.mid')); len(song_mid)

19876

In [11]:
song_json[0]

PosixPath('../data/midi/hooktheory/event/w/wayne-sharpe/yu-gi-oh-theme-song/chorus_symbol_key.json')

In [12]:
def get_hooktheory_attr(song_json_path, key2info):
    song_json = json.load(open(song_json_path, 'r'))
    metadata = song_json['metadata']
    song_info = key2info[song_key(song_json_path)].copy()
    artist = song_json_path.parts[-3]
    title = song_json_path.parts[-2]
    section = song_json_path.name.split('_')[0]
    midi_path = str(song_json_path.with_suffix('.mid')).replace('event', 'pianoroll').replace('symbol_', '')
    
    # convert stream here
    metadata = {
        'artist': artist,
        'section': section,
        'original_path': midi_path,
        'parts': song_info['section'],
        'song_url': song_info['song_url'],
        'genres': song_info['genres'],
        'midi_title': metadata['title'],
        'ht_bpm': metadata['BPM'],
        'ht_mode': metadata['mode'],
        'ht_key': metadata['key'],
        'ht_time_signature': metadata['beats_in_measure']
    }
#     try:
    mode = metadata['ht_mode']
    if mode is None:
        print('No mode found. Assuming major', song_json_path)
        mode = 'major'
    offset = keyc_offset(metadata['ht_key'], mode)
    stream_info = get_music21_attr(midi_path, transpose=True, offset=offset)
#     except Exception as e:
#         stream_info = {}
#         print('Could not parse stream', midi_path, e)
    return {**metadata, **stream_info}

In [13]:
# sanity check
# hook_out = get_hooktheory_attr(song_json[1000], key2info); hook_out

In [14]:
file2metadata = load_json(hook_metapath)
if file2metadata is None: file2metadata = {}

In [19]:
subset = song_json
for s in tqdm(subset, total=len(subset)):
    key = ''.join(s.with_suffix('').parts[-3:])
    if key in file2metadata: continue
    song_attr = get_hooktheory_attr(s, key2info)
    file2metadata[key] = song_attr


 64%|██████▍   | 12704/19876 [00:00<00:00, 77546.71it/s]

Could not parse stream ../data/midi/hooktheory/pianoroll/t/toby-fox/determination/pre-chorus-and-chorus_key.mid list index out of range



 65%|██████▌   | 12923/19876 [00:34<16:48,  6.90it/s]

Could not parse stream ../data/midi/hooktheory/pianoroll/t/the-meters/what-cha-say/intro-and-verse_key.mid list index out of range



 65%|██████▌   | 12947/19876 [00:37<14:22,  8.03it/s]

...


 99%|█████████▉| 19708/19876 [18:11<00:22,  7.38it/s]

No mode found. Assuming major ../data/midi/hooktheory/event/a/augustana/boston/verse_symbol_key.json



 99%|█████████▉| 19733/19876 [18:15<00:19,  7.36it/s]

Could not parse stream ../data/midi/hooktheory/pianoroll/a/asf/asdfas/intro-and-verse_key.mid list index out of range



100%|██████████| 19876/19876 [18:38<00:00,  5.44it/s]


In [18]:
save_json(file2metadata, hook_metapath)

Save song_list

In [20]:
json.dump(hooktheory_list, open(song_list_path, 'w'))

In [21]:
hooktheory_list[-3:]

[{'artist': 'apink',
  'section': 'chorus',
  'original_path': '../data/midi/hooktheory/pianoroll/a/apink/only-one/chorus_key.mid',
  'parts': ['chorus'],
  'song_url': 'https://www.hooktheory.com/theorytab/view/apink/only-one',
  'genres': ['K-pop'],
  'title': 'Only One',
  'mode': '1',
  'instruments': ['Piano', 'Piano'],
  'bpm': 102.0,
  'inferred_key': 'F# major',
  'seconds': 18.823529411764707,
  'time_signature': '4/4',
  'inferred_keyc': 'C major',
  'midi_keyc': '../data/midi/transposed/hooktheory/pianoroll/a/apink/only-one/chorus_key.mid'},
 {'artist': 'apink',
  'section': 'chorus',
  'original_path': '../data/midi/hooktheory/pianoroll/a/apink/promise-u/chorus_key.mid',
  'parts': ['chorus'],
  'song_url': 'https://www.hooktheory.com/theorytab/view/apink/promise-u',
  'genres': ['K-pop'],
  'title': 'pu',
  'mode': '1',
  'instruments': ['Piano', 'Piano'],
  'bpm': 70.0,
  'inferred_key': 'A- major',
  'seconds': 27.428571428571427,
  'time_signature': '4/4',
  'inferred_k

In [22]:
hooktheory_list = json.load(open(song_list_path, 'r'))

### Gather FreeMidi

In [10]:
freemidi_path = path/'freemidi_scrape'; freemidi_path.ls()

[PosixPath('../data/midi/freemidi_scrape/genre-disco'),
 PosixPath('../data/midi/freemidi_scrape/genre-pop'),
 PosixPath('../data/midi/freemidi_scrape/genre-dance-eletric'),
 PosixPath('../data/midi/freemidi_scrape/genre-punk'),
 PosixPath('../data/midi/freemidi_scrape/genre-hip-hop-rap'),
 PosixPath('../data/midi/freemidi_scrape/genre-rock')]

In [29]:
def parse_freemidi_songs(fp, genre=None):
    metadata,attr = {},{}
    try:
        name = fp.with_suffix('').name.split(' - ')
        artist = name[0]
        title = name[-1]
        metadata = {
            'artist': artist,
            'title': title,
            'midi': str(fp),
            'genre': genre
        }
    except Exception as e:
        print('Exeption:', fp, e)
    try:
        attr = get_music21_attr(fp)
    except Exception as e:
        print('Exeption:', fp, e)
    return {**metadata, **attr}

In [12]:
def parse_freemidi_genre(genre_path, genre):
    file_list = genre_path.ls()
    parsed_songs = [parse_freemidi_songs(fp, genre) for fp in tqdm(file_list, total=len(file_list))]
    
    metadata_path = genre_path.parent/f'freemidi_{genre}_metadata.json'
    json.dump(parsed_songs, open(metadata_path, 'w'))
    
    return parsed_songs, metadata_path

In [15]:
dance_files = (freemidi_path/'genre-dance-eletric/').ls()

In [23]:
# sanity check
# freemidi_out = parse_freemidi_songs(dance_files[0], genre='dance'); freemidi_out

In [30]:
freemidi_dance_list, freemidi_dance_path = parse_freemidi_genre(freemidi_path/'genre-dance-eletric', 'dance')





  0%|          | 0/1122 [00:00<?, ?it/s]

Transposed file exists. Skipping:  ../data/midi/transposed/freemidi_scrape/genre-dance-eletric/Vengaboys - Up And Down.mid






  0%|          | 1/1122 [00:01<25:21,  1.36s/it]

Transposed file exists. Skipping:  ../data/midi/transposed/freemidi_scrape/genre-dance-eletric/ATB - Dont stop.mid






  0%|          | 2/1122 [00:03<29:47,  1.60s/it]

...





 93%|█████████▎| 1044/1122 [2:42:36<18:27, 14.20s/it]



 93%|█████████▎| 1045/1122 [2:42:41<14:54, 11.62s/it]



 93%|█████████▎| 1046/1122 [2:42:55<15:35, 12.31s/it]



 93%|█████████▎| 1047/1122 [2:42:56<11:01,  8.82s/it]



 93%|█████████▎| 1048/1122 [2:42:58<08:21,  6.78s/it]



 93%|█████████▎| 1049/1122 [2:42:59<06:17,  5.17s/it]



 94%|█████████▎| 1050/1122 [2:43:04<06:08,  5.12s/it]



 94%|█████████▎| 1051/1122 [2:43:08<05:44,  4.86s/it]



 94%|█████████▍| 1052/1122 [2:43:18<07:22,  6.32s/it]



 94%|█████████▍| 1053/1122 [2:43:28<08:39,  7.53s/it]



 94%|█████████▍| 1054/1122 [2:43:41<10:13,  9.02s/it]



 94%|█████████▍| 1055/1122 [2:43:51<10:16,  9.20s/it]

Exeption: ../data/midi/freemidi_scrape/genre-dance-eletric/New Order - Ceremony.mid cannot get a seconds duration when no TempoIndication classes are found in or before this Stream.






 94%|█████████▍| 1056/1122 [2:43:59<09:46,  8.89s/it]



 94%|█████████▍| 1057/1122 [2:44:16<12:16, 11.33s/it]



 94%|█████████▍| 1058/1122 [2:44:21<10:15,  9.61s/it]



 94%|█████████▍| 1059/1122 [2:44:31<10:02,  9.57s/it]



 94%|█████████▍| 1060/1122 [2:44:33<07:30,  7.27s/it]



 95%|█████████▍| 1061/1122 [2:45:02<14:10, 13.94s/it]



 95%|█████████▍| 1062/1122 [2:45:18<14:33, 14.57s/it]



 95%|█████████▍| 1063/1122 [2:45:19<10:20, 10.51s/it]



 95%|█████████▍| 1064/1122 [2:45:32<10:47, 11.17s/it]



 95%|█████████▍| 1065/1122 [2:45:36<08:35,  9.05s/it]



 95%|█████████▌| 1066/1122 [2:45:45<08:25,  9.03s/it]



 95%|█████████▌| 1067/1122 [2:45:51<07:19,  8.00s/it]



 95%|█████████▌| 1068/1122 [2:45:58<06:57,  7.74s/it]



 95%|█████████▌| 1069/1122 [2:46:04<06:20,  7.18s/it]



 95%|█████████▌| 1070/1122 [2:46:06<05:00,  5.77s/it]



 95%|█████████▌| 1071/1122 [2:46:08<04:00,  4.72s/it]



 96%|█████████▌| 1072/1122 [2:46:12<03:31,  4.22s/it]



 96%|█████████▌| 1073/1122 

Exeption: ../data/midi/freemidi_scrape/genre-dance-eletric/Tatu - Robot.mid badly formated midi bytes, got: b'RIFF\x82N\x00\x00RMIDdata\x1cN\x00\x00'






 98%|█████████▊| 1094/1122 [2:48:20<02:50,  6.08s/it]



 98%|█████████▊| 1095/1122 [2:48:25<02:37,  5.84s/it]



 98%|█████████▊| 1096/1122 [2:48:32<02:39,  6.14s/it]



 98%|█████████▊| 1097/1122 [2:48:36<02:18,  5.53s/it]



 98%|█████████▊| 1098/1122 [2:48:39<01:51,  4.65s/it]



 98%|█████████▊| 1099/1122 [2:48:44<01:52,  4.90s/it]



 98%|█████████▊| 1100/1122 [2:48:47<01:35,  4.33s/it]



 98%|█████████▊| 1101/1122 [2:48:49<01:11,  3.39s/it]



 98%|█████████▊| 1102/1122 [2:48:56<01:32,  4.64s/it]



 98%|█████████▊| 1103/1122 [2:49:21<03:20, 10.56s/it]



 98%|█████████▊| 1104/1122 [2:49:33<03:20, 11.12s/it]



 98%|█████████▊| 1105/1122 [2:49:53<03:56, 13.92s/it]



 99%|█████████▊| 1106/1122 [2:49:57<02:52, 10.77s/it]



 99%|█████████▊| 1107/1122 [2:50:00<02:08,  8.55s/it]



 99%|█████████▉| 1108/1122 [2:50:04<01:40,  7.20s/it]



 99%|█████████▉| 1109/1122 [2:50:11<01:31,  7.04s/it]



 99%|█████████▉| 1110/1122 [2:50:11<01:00,  5.08s/it]



 99%|█████████▉| 1111/1122 

In [32]:
freemidi_dance_path = freemidi_path/'freemidi_dance_metadata.json'

'freemidi_dance_metadata.json'

In [34]:
freemidi_dance_list_load = json.load(open(freemidi_dance_path, 'r'))

In [ ]:
freemidi_pop_list, freemidi_pop_path = parse_freemidi_genre(freemidi_path/'genre-pop', 'pop')

### Gather Cprato

### Yamaha - piano

In [ ]:
ecomp_path = Path('../data/midi/piano_e_competition')
ecomp_scrape_path = Path('../data/midi/ecomp_midi')

### Classic Piano

In [ ]:
classic_scrape_path = Path('../data/midi/classic_piano_scrape')

### Creating CSV

In [ ]:
import csv
toCSV = [{'name':'bob','age':25,'weight':200},
         {'name':'jim','age':31,'weight':180}]
keys = toCSV[0].keys()
with open('people.csv', 'wb') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(toCSV)

## Preparing the data

In [3]:
class MusicTokenizer():
    def __init__(self):
        super().__init__()
        self.n_cpus = num_cpus()
        
    def process_text(self, t:str) -> List[str]:
        return t.split(" ")
    
    def _process_all_1(self, texts:Collection[str]) -> List[List[str]]:
        return [self.process_text(t) for t in texts]

    def process_all(self, texts:Collection[str]) -> List[List[str]]:
        "Process a list of `texts`."
        if self.n_cpus <= 1: return self._process_all_1(texts)
        with ProcessPoolExecutor(self.n_cpus) as e:
            return sum(e.map(self._process_all_1, partition_by_cores(texts, self.n_cpus)), [])


In [4]:
path = Path('data/composers/notewise/piano_solo/note_range62/sample_freq12')
path.ls()[:5]

[PosixPath('data/composers/notewise/piano_solo/note_range62/sample_freq12/jazz'),
 PosixPath('data/composers/notewise/piano_solo/note_range62/sample_freq12/ravel'),
 PosixPath('data/composers/notewise/piano_solo/note_range62/sample_freq12/bach'),
 PosixPath('data/composers/notewise/piano_solo/note_range62/sample_freq12/schumann'),
 PosixPath('data/composers/notewise/piano_solo/note_range62/sample_freq12/liszt')]

In [5]:
bs=32

In [6]:
if (path/'tmp/itos.pkl').exists():
# if False:
    data = TextLMDataBunch.load(path, bs=bs)
else:
    p = [OpenFileProcessor(), TokenizeProcessor(tokenizer=MusicTokenizer(), chunksize=10), NumericalizeProcessor(vocab=None, max_vocab=500)]

    data = (TextList.from_folder(path, recurse=True, processor=p)
            .random_split_by_pct(0.05, seed=6)
            .label_for_lm()
            .databunch(bs=bs))
    data.save('tmp')
vocab = data.train_ds.vocab
len(data.train_ds), len(data.valid_ds), len(data.train_ds.vocab.itos)

(1797, 94, 158)

In [7]:
t = data.train_ds[0][0]
t.text[:50], t.data

('xxbos wait25 wait25 wait25 wait25 wait25 wait25 wa',
 array([  2, 124, 124, 124, ...,   9, 105,   9, 157]))

In [8]:
data.show_batch()

idx,text
0,xxbos p21 p28 p36 wait7 endp36 wait2 p29 p36 wait6 endp28 wait25 endp29 endp36 wait1 endp21 wait1 p28 wait2 p27 endp28 wait2 endp27 wait2 p20 p28 wait18 p29 p35 wait14 endp28 wait6 endp29 endp35 wait2 p28 wait5 endp28 wait2 endp20 wait1 p19 p28 wait7 endp28 wait4 endp19 wait7 p40 p47 wait2 p35 p36 p43 wait2 endp43 wait9 endp35 wait2 endp40 endp47 wait2 endp36 wait5 p33 p36 p40 p45 wait2 endp33
1,p24 wait8 endp24 wait4 p24 wait5 endp24 wait1 p20 wait3 endp20 wait3 p20 wait1 endp34 wait1 endp25 endp30 wait4 p36 wait2 endp20 endp36 wait4 p13 p20 p24 endp24 p28 p31 p36 wait2 endp28 endp31 endp36 wait1 endp20 wait3 p22 p27 p32 p37 wait2 endp32 endp37 wait1 endp22 endp27 wait3 p13 p23 p28 p32 wait2 endp23 wait4 p23 endp23 wait1 endp13 wait1 endp28 wait4 p6 p22 p28 wait6 p13 wait1 endp22
2,wait1 endp27 wait2 endp34 wait1 p39 p43 wait3 endp39 endp43 wait2 p22 p27 p31 wait3 endp22 endp27 endp31 wait1 endp19 wait2 p34 wait2 endp34 wait2 endp46 wait2 p39 endp39 p43 endp43 wait9 p14 p38 wait6 p26 wait1 endp14 wait2 endp26 wait1 p33 wait1 endp38 wait3 p7 p19 p39 wait1 endp33 wait2 endp7 wait1 p27 wait1 endp19 wait2 endp27 wait3 p34 wait3 endp34 p43 wait6 p27 wait1 endp43 wait1 endp27 wait1
3,p4 p35 wait3 endp35 wait1 p32 wait2 endp16 wait1 endp4 p9 endp32 p33 wait6 p12 wait1 endp9 wait6 endp12 wait2 p16 wait6 p21 wait1 endp16 wait5 endp33 wait1 endp21 wait1 p9 p16 p40 wait7 endp40 p45 wait6 p9 p21 wait1 endp16 wait4 endp45 wait1 p36 wait3 p35 endp36 wait3 p33 endp35 wait2 endp9 endp21 wait1 p20 p23 endp33 p35 wait4 p44 wait1 endp35 wait2 endp44 wait1 p47 wait7 p16 p20
4,endp52 endp56 wait6 p44 p47 p52 p56 wait1 p23 p28 p32 p35 wait3 endp35 wait2 endp28 endp32 wait2 endp47 endp56 wait1 endp23 wait2 endp44 endp52 wait14 p4 wait2 endp4 wait10 p52 wait3 p24 p28 p31 p36 p40 wait2 endp24 endp28 endp31 endp36 endp40 endp52 wait6 p24 endp24 p28 p31 p36 p40 p50 wait2 endp28 endp31 endp36 wait1 endp40 endp50 wait4 p24 endp24 p28 p31 p36 p40 p48 wait2 endp28 endp31
